In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Build Elo

In [25]:
from simulation_utils import db_get_data_for_latest_season

# Get data for the latest season
df = db_get_data_for_latest_season()

df

,season,home,away,utc_date,status,matchday,home_score,away_score,home_outcome,away_outcome,home_manager,away_manager,home_manager_count,away_manager_count,home_position,away_position
257,2024,Manchester United FC,Fulham FC,2024-08-16 19:00:00,TIMED,1,None,None,None,None,Erik ten Hag,Marco Silva,0,0,2,1
256,2024,Ipswich Town FC,Liverpool FC,2024-08-17 11:30:00,TIMED,1,None,None,None,None,Kieran McKenna,Arne Slot,0,0,2,3
255,2024,Arsenal FC,Wolverhampton Wanderers FC,2024-08-17 14:00:00,TIMED,1,None,None,None,None,Mikel Arteta,Gary O'Neil,0,0,2,12
254,2024,Everton FC,Brighton & Hove Albion FC,2024-08-17 14:00:00,TIMED,1,None,None,None,None,Sean Dyche,Fabian Hürzeler,0,0,4,3
253,2024,Newcastle United FC,Southampton FC,2024-08-17 14:00:00,TIMED,1,None,None,None,None,Eddie Howe,Russell Martin,0,0,9,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,2024,Ipswich Town FC,West Ham United FC,2025-05-25 15:00:00,TIMED,38,None,None,None,None,Kieran McKenna,Julen Lopetegui,0,0,12,3
126,2024,Fulham FC,Manchester City FC,2025-05-25 15:00:00,TIMED,38,None,None,None,None,Marco Silva,Pep Guardiola,0,0,13,9
127,2024,AFC Bournemouth,Leicester City FC,2025-05-25 15:00:00,TIMED,38,None,None,None,None,Andoni Iraola,Steve Cooper,0,0,1,11
353,2024,Tottenham Hotspur FC,Brighton & Hove Albion FC,2025-05-25 15:00:00,TIMED,38,None,None,None,None,Ange Postecoglou,Fabian Hürzeler,0,0,4,17


## Test Against 2023-2024 Season

### Predict 2023-2024 Season

In [28]:
from simulation_utils import build_elo_before_season

# Build the elo dataframe before the current season
elo_df = build_elo_before_season(df)
elo_df

,elo,club_value,normalized_club_value,exponential_club_value,normalized_exponential_club_value,adjusted_elo
team,,,,,,
Manchester City FC,1691.433730,1.408200e+09,1.000000,1.718282,1.000000,1991.433730
Arsenal FC,1677.315272,1.172000e+09,0.820585,1.271828,0.740174,1899.367589
Chelsea FC,1611.633389,1.090200e+09,0.758450,1.134965,0.660523,1809.790415
Liverpool FC,1616.126787,9.493000e+08,0.651424,0.918271,0.534412,1776.450477
Manchester United FC,1537.082690,8.181500e+08,0.551804,0.736383,0.428558,1665.649959
Aston Villa FC,1564.405613,6.552000e+08,0.428029,0.534230,0.310910,1657.678502
Tottenham Hotspur FC,1521.711361,7.937000e+08,0.533232,0.704432,0.409963,1644.700307
Newcastle United FC,1550.106306,6.566500e+08,0.429130,0.535921,0.311894,1643.674386
Crystal Palace FC,1563.087537,4.370000e+08,0.262286,0.299899,0.174534,1615.447760


In [33]:
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
import joblib
from tqdm import tqdm
from functools import partial

from simulation_utils import download_best_params_from_s3, download_model_and_scaler_from_s3, get_elo_dict_from_df, simulate_and_get_results

# Get the model and scalar from S3
model_file = Path("random_forest.joblib")
scaler_file = Path("standard_scaler.joblib")
download_model_and_scaler_from_s3(model_file, scaler_file)
model = joblib.load(model_file)
scaler = joblib.load(scaler_file)

# Get adjusted Elo dict
adjusted_elo = get_elo_dict_from_df(elo_df)

# Number of simulations to run
num_simulations = 10
# num_simulations = 1000

# Initialize a list to store results
seasons = []

# Get the best parameters for the model
best_params = download_best_params_from_s3()

# Create a partial function to pass the same arguments to each simulation
simulate_and_get_results_partial = partial(simulate_and_get_results, df=df, elo=adjusted_elo, model=model, scaler=scaler, k=best_params.k, half_life=best_params.half_life, decay_method=best_params.decay_method)

# Initialize a pool of workers
with ProcessPoolExecutor() as executor:
    seasons = list(tqdm(executor.map(simulate_and_get_results_partial, range(num_simulations)),  total=num_simulations, desc='Simulating', unit='season'))

# Unlink the model and scaler files
model_file.unlink()
scaler_file.unlink()

Model downloaded to random_forest.joblib
Scaler downloaded to standard_scaler.joblib


Simulating: 100%|██████████| 10/10 [00:13<00:00,  1.37s/season]


### Analyze Results Compared to actual 2023-2024 Season

### Post Process Simulation Results

In [34]:
import pandas as pd

# Aggregate the results
results = pd.concat(seasons).groupby("team").agg({"total_outcome": "sum"})

# Sort results based on total outcome
results = results.sort_values("total_outcome", ascending=False)

# Get the place each team finished in the league
results["place"] = range(1, len(results) + 1)

# For each season, get the place each team finished in the league
for index, season_df in enumerate(seasons):
    season_df["place"] = range(1, len(season_df) + 1)

# Get the average place each team finished in the league
average_results = pd.concat(seasons).groupby("team").agg({"place": "mean"}).sort_values("place")

# Get the season from seasons
list_of_seasons = df["season"].unique()

assert len(list_of_seasons) == 1, "All seasons should be the same"

# Get the season
simulation_season = list_of_seasons[0]

# Get a mapping of team names to a list places they finished in the league
team_place_mapping = {}
for team in average_results.index:
    team_place_mapping[team] = [season_df.loc[team, "place"] for season_df in seasons]

# Get the total number of seasons simulated
total_seasons = len(seasons)

# Get a mapping of times each team won the league
team_win_mapping = {}
for team in average_results.index:
    team_win_mapping[team] = sum([season_df.loc[team, "place"] == 1 for season_df in seasons]) / total_seasons

# Get a mapping of times each team finished in the top 4
team_top_4_mapping = {}
for team in average_results.index:
    team_top_4_mapping[team] = sum([season_df.loc[team, "place"] <= 4 for season_df in seasons]) / total_seasons

# Get a mapping of times each team finished in the bottom 3
team_bottom_3_mapping = {}
for team in average_results.index:
    team_bottom_3_mapping[team] = sum([season_df.loc[team, "place"] >= len(season_df) - 3 for season_df in seasons]) / total_seasons

# Build a dataframe with the average place, times won, times in top 4, and times in bottom 3
average_results["win_premier_league"] = [team_win_mapping[team] for team in average_results.index]
average_results["top_4"] = [team_top_4_mapping[team] for team in average_results.index]
average_results["bottom_3"] = [team_bottom_3_mapping[team] for team in average_results.index]

# Show the dataframe
average_results

,place,win_premier_league,top_4,bottom_3
team,,,,
Manchester City FC,3.8,0.2,0.7,0.0
Arsenal FC,4.0,0.2,0.7,0.0
Chelsea FC,4.3,0.2,0.6,0.0
Aston Villa FC,5.4,0.2,0.3,0.0
Crystal Palace FC,6.5,0.1,0.3,0.0
Manchester United FC,6.9,0.1,0.3,0.0
Newcastle United FC,7.5,0.0,0.3,0.0
Tottenham Hotspur FC,7.6,0.0,0.3,0.0
Liverpool FC,8.1,0.0,0.1,0.0


### Find the Positions of the Teams in the 2023-2024 Season for all Seasons

In [32]:
team_to_position = {team: {} for team in df["home"].unique()}
for season in seasons:
    # Assign the position of each team
    for i, team in enumerate(season.index, 1):
        if i not in team_to_position[team]:
            team_to_position[team][i] = 0
        team_to_position[team][i] += 1

# Create a dataframe where each row is a team and the one column is an array of positions in which the team finished in the league
team_positions_df = pd.DataFrame.from_dict(team_to_position, orient="index").fillna(0).astype(int)

# Stack the dataframe to get a row for each team-position pair
team_positions_df = team_positions_df.stack().reset_index().rename(columns={'level_0': 'team', 'level_1': 'position', 0: 'count'})

# Set the index to the team name-position pair
team_positions_df.set_index(["team", "position"], inplace=True)

# Sort the position index
team_positions_df = team_positions_df.sort_index()

team_positions_df

count
team                       position       
AFC Bournemouth            1             0
                           2             0
                           3             0
                           4             0
                           5             1
...                                    ...
Wolverhampton Wanderers FC 16            1
                           17            0
                           18            1
                           19            1
                           20            0

[400 rows x 1 columns]

## Store the Data

In [11]:
from simulation_utils import db_store_results

# Store the results in the database
db_store_results(simulation_season, average_results, team_positions_df)